## Clone, install, import

In [ ]:
!git clone #add link

In [1]:
%pip install -r requirements.txt -q

import ucto
ucto.installdata()

Note: you may need to restart the kernel to use updated packages.


Uctodata configuration directory already exists: /home/arjan_v_d/.config/ucto, refusing to overwrite, please remove it first if you want to install all data anew.


In [1]:
from preprocess.add_html_declaration import *
from preprocess.dbnl_xml_to_txt import *
#from preprocess.folia_to_df import *
import ucto
import os
#from preprocess.txt_to_folia import *
import tqdm
import pandas as pd

## Download

In [ ]:
from utils.create_if_absent import create_if_absent
from download.download_files import download_and_unzip

create_if_absent("dbnl_xml")
download_and_unzip(url = "https://www.dbnl.org/letterkunde/pd/xml_pd.zip", output_dir = "dbnl_xml")

## Preprocess

In [24]:
html_to_decimal_dict = make_html_entity_dict()
addition =  additional_declaration_str(html_to_decimal_dict, '.dtd"')
add_declaration_to_xml('dbnl_xml', '<!DOCTYPE', '.dtd"', addition)

In [25]:
create_if_absent("dbnl_txt")
dbnl_to_txt("dbnl_xml", "dbnl_txt")

Directory dbnl_txt created
file teen002bijz01 could not be parsed
file hugo001piad01 could not be parsed
file vaen001quin01 could not be parsed
file vry_001anat01 could not be parsed
file _gid001183901 could not be parsed
file luyk001jezu01 could not be parsed
file aa__001biog02 could not be parsed
file herl001besc01 could not be parsed
file lede003drie01 could not be parsed
file elge001zinn01 could not be parsed
file ferm001nieu01 could not be parsed
file leuv001amor01 could not be parsed
file aa__001biog11 could not be parsed
file carp002verh01 could not be parsed


In [10]:
create_if_absent("dbnl_folia")
txt_to_folia("dbnl_txt", "dbnl_folia")

Directory dbnl_folia already exists


looping through text files: 100%|██████████| 3415/3415 [38:02<00:00,  1.50it/s]  


In [2]:

import folia.main as folia
import pandas as pd
import os
import random 
from planimals.preprocess.dbnl_xml_to_txt import extract_dbnl_id

def list_folia_files(directory):
  """ 
  List all folia files in a certain directory.
  """
  folia_files = []
  for filename in os.listdir(directory):
    if filename.endswith('folia.xml'):
      folia_files.append(filename)
  return folia_files

def split_words(sentence):
  sentence_lst = []
  for w in sentence.words():
    sentence_lst.append(w.text())
  return sentence_lst

def split_folia(doc):
  doc_lst = []
  for s in doc.sentences():
    s_lst = split_words(s)
    doc_lst.append(s_lst)
  return doc_lst

def select_random_files(path_to_folder, file_extension, num_files):
  file_paths = [os.path.join(path_to_folder, f) for f in os.listdir(path_to_folder) if f.endswith(file_extension)]
  return random.sample(file_paths, num_files)
  
def select_all_files(path_to_folder, file_extension):
  return [os.path.join(path_to_folder, f) for f in os.listdir(path_to_folder) if f.endswith(file_extension)]


def folia_to_df(folia_path):
  doc = folia.Document(file=folia_path)
  doc_lst = split_folia(doc)
  df = pd.DataFrame({'sentence': doc_lst})
  df["text_id"] = extract_dbnl_id(folia_path)
  return df

def extract_dbnl_id(f_name):
    """
    extract dbnl-id from a filename.
    """
    pattern = r"[a-zA-Z_]{4}\d{3}[a-zA-Z_\d]{4}\d{2}"
    regex = re.compile(pattern)
    return regex.search(f_name)[0]

def folia_to_df_with_sentence(folia_path):
  out_path = f"dbnl_sentences/{extract_dbnl_id(folia_path)}.pkl"
  if not os.path.exists(out_path):
    try:
        doc = folia.Document(file=folia_path)
        splitted_s_lst = split_folia(doc)
        s_lst = split_folia_in_sentences(doc)
        df = pd.DataFrame({'sentence': s_lst , 'splitted':splitted_s_lst})
        df["text_id"] = extract_dbnl_id(folia_path)
        df.to_pickle(out_path)
    except:
        print(f"for{folia_path} it didn't work")
  return

def split_folia_in_sentences(doc):
  doc_lst = []
  for s in doc.sentences():
    doc_lst.append(s.text())
  return doc_lst


In [11]:
create_if_absent("dbnl_sentences")
folia_files = select_all_files("dbnl_folia", "folia.xml")

for f in tqdm.tqdm(folia_files, desc = "making df"):
  folia_to_df_with_sentence(f)



Directory dbnl_sentences already exists


making df:   0%|          | 0/3415 [00:00<?, ?it/s]

fordbnl_folia/_sta001stat02.folia.xml it didn't work


making df: 100%|██████████| 3415/3415 [01:25<00:00, 39.84it/s]

fordbnl_folia/_vor003vors01.folia.xml it didn't work


In [ ]:
# 3 things: text in sentence, dutch, footnotes

In [4]:
import os

def count_files_in_folder(folder_path):
    file_count = 0

    # Iterate over all the items in the folder
    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

# Provide the path to the folder you want to count files in
folder_paths = ['dbnl_xml', 'dbnl_txt', 'dbnl_folia', 'dbnl_sentences']

for p in folder_paths:
    num_files = count_files_in_folder(p)
    print(f"Number of files in '{p}': {num_files}")


Number of files in 'dbnl_xml': 3433
Number of files in 'dbnl_txt': 3415
Number of files in 'dbnl_folia': 3422
Number of files in 'dbnl_sentences': 3413


In [1]:
subfolders1 = ["towards_big_df/subfolder_1", "towards_big_df/subfolder_2", "towards_big_df/subfolder_3", "towards_big_df/subfolder_4", "towards_big_df/subfolder_5"]
subfolders2 = ["towards_big_df/subfolder_5", "towards_big_df/subfolder_6", "towards_big_df/subfolder_7"]


In [ ]:
import os
import tqdm
import pandas as pd

def select_all_files(path_to_folder, file_extension):
  return [os.path.join(path_to_folder, f) for f in os.listdir(path_to_folder) if f.endswith(file_extension)]

n = 0
for s in subfolders1:
    n+=1
    paths = select_all_files(s, ".pkl")
    dfs = []
    for p in tqdm.tqdm(paths, desc = "reading pkl's"):
        df = pd.read_pickle(p)
        dfs.append(df)
    total_df = pd.concat(dfs, ignore_index = True)
    total_df.to_pickle(f"towards_big_df_sub_{n}")

reading pkl's: 100%|██████████| 500/500 [00:13<00:00, 38.28it/s] 

In [2]:
import os
import tqdm
import pandas as pd

def select_all_files(path_to_folder, file_extension):
  return [os.path.join(path_to_folder, f) for f in os.listdir(path_to_folder) if f.endswith(file_extension)]

n = 5
for s in subfolders2:
    n+=1
    paths = select_all_files(s, ".pkl")
    dfs = []
    for p in tqdm.tqdm(paths, desc = "reading pkl's"):
        df = pd.read_pickle(p)
        dfs.append(df)
    total_df = pd.concat(dfs, ignore_index = True)
    total_df.to_pickle(f"towards_big_df_sub_{n}")

reading pkl's: 100%|██████████| 413/413 [00:11<00:00, 36.02it/s]


In [1]:
import os
import shutil

# Define the source folder containing the files
source_folder = "dbnl_sentences"

# Define the destination folder for the subfolders
destination_folder = "towards_big_df"

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Get a list of all the files in the source folder
files = os.listdir(source_folder)

# Calculate the number of files per subfolder
files_per_subfolder = 500

# Calculate the total number of subfolders needed
total_subfolders = len(files) // files_per_subfolder + 1

# Create the subfolders and move the files into them
for i in range(total_subfolders):
    # Create a subfolder
    subfolder_path = os.path.join(destination_folder, f'subfolder_{i+1}')
    os.makedirs(subfolder_path, exist_ok=True)
    
    # Move the files into the subfolder
    start_index = i * files_per_subfolder
    end_index = start_index + files_per_subfolder
    for file in files[start_index:end_index]:
        file_path = os.path.join(source_folder, file)
        shutil.move(file_path, subfolder_path)


In [7]:
df1 = "towards_big_df_sub_1"
df2 = "towards_big_df_sub_2"
df3 = "towards_big_df_sub_3"
df4 = "towards_big_df_sub_4"
df5 = "towards_big_df_sub_5"
df6 = "towards_big_df_sub_6"
df7 = "towards_big_df_sub_7"


In [8]:
import pandas as pd
import fastparquet
e = df5
f = "df5" 
e_df = pd.read_pickle(e)
e_df.to_parquet(f'parquets/{f}.parquet')

In [ ]:
merged_df = pd.concat([
    pd.read_parquet('parquets/df1.parquet'),
    pd.read_parquet('parquets/df2.parquet'),
    pd.read_parquet('parquets/df3.parquet'),
    pd.read_parquet('parquets/df4.parquet'),
    pd.read_parquet('parquets/df5.parquet'),
    pd.read_parquet('parquets/df6.parquet'),
    pd.read_parquet('parquets/df7.parquet')],
    ignore_index=True)


In [4]:
import pandas as pd
import fastparquet
merged_df = pd.concat([
    pd.read_parquet('parquets/df1.parquet'),
    pd.read_parquet('parquets/df2.parquet')],
    ignore_index=True)

In [ ]:
merged_df.to_parquet("parquets/df1and2.parquet")

In [3]:
import pandas as pd
a = pd.read_parquet("parquets/df1and2.parquet")

TypeError: 'NoneType' object is not iterable